In [ ]:
import os
import csv
import datetime as dt
from concurrent.futures import ThreadPoolExecutor, as_completed
from authorizenet import apicontractsv1
from authorizenet.apicontrollers import getCustomerProfileIdsController, getCustomerProfileController
from authorizenet.constants import constants

# --- Set credentials via environment variables ---
LOGIN_ID = os.environ.get("AUTHNET_LOGIN_ID")
TRANS_KEY = os.environ.get("AUTHNET_TRANSACTION_KEY")

if not LOGIN_ID or not TRANS_KEY:
    raise SystemExit("Missing credentials. Set AUTHNET_LOGIN_ID and AUTHNET_TRANSACTION_KEY in your environment.")

# --- Choose environment ---
ENV = constants.PRODUCTION

# Save to Documents folder where the script is located
CSV_FILE = os.path.join(os.path.dirname(__file__), "expired_cards_output.csv")
PROGRESS_FILE = os.path.join(os.path.dirname(__file__), "scan_progress.txt")

# Number of concurrent threads
MAX_WORKERS = 10

# Batch size - save progress every N profiles
BATCH_SIZE = 5000  # Save every 5000 profiles

# --- Helper functions ---
def parse_expiration(exp):
    """Return (year, month) tuple from common formats MMYY, MM/YY, YYYY-MM, MM-YYYY"""
    s = str(exp) if exp is not None else ""
    s = s.strip()
    
    if not s or s == "XXXX":
        return None
    try:
        if "-" in s:
            parts = s.split("-")
            if len(parts[0]) == 4:
                return int(parts[0]), int(parts[1])
            else:
                return int(parts[1]), int(parts[0])
        if "/" in s:
            m, y = s.split("/")
            year = int("20" + y) if len(y) == 2 else int(y)
            return year, int(m)
        if len(s) == 4 and s.isdigit():
            return int("20" + s[2:]), int(s[:2])
    except (ValueError, IndexError):
        return None
    return None

def is_expired(exp_date):
    """Check if a credit card is expired based on expiration date string"""
    today = dt.date.today()
    parsed = parse_expiration(exp_date)
    if not parsed:
        return False
    y, m = parsed
    
    try:
        if m == 12:
            next_month = dt.date(y + 1, 1, 1)
        else:
            next_month = dt.date(y, m + 1, 1)
        last_day = next_month - dt.timedelta(days=1)
        
        return today > last_day
    except ValueError:
        return False

# --- Fetch all customer profile IDs ---
def fetch_all_profile_ids():
    """Retrieve all customer profile IDs from Authorize.Net"""
    request = apicontractsv1.getCustomerProfileIdsRequest()
    request.merchantAuthentication = apicontractsv1.merchantAuthenticationType()
    request.merchantAuthentication.name = LOGIN_ID
    request.merchantAuthentication.transactionKey = TRANS_KEY

    controller = getCustomerProfileIdsController(request)
    controller.setenvironment(ENV)
    controller.execute()
    response = controller.getresponse()

    if response.messages.resultCode != "Ok":
        raise RuntimeError(f"Error fetching profile IDs: {response.messages.message[0].text}")

    return response.ids.numericString or []

# --- Fetch profile details with unmasked expiration dates ---
def fetch_profile(profile_id):
    """Fetch detailed customer profile information with unmaskExpirationDate enabled"""
    request = apicontractsv1.getCustomerProfileRequest()
    request.merchantAuthentication = apicontractsv1.merchantAuthenticationType()
    request.merchantAuthentication.name = LOGIN_ID
    request.merchantAuthentication.transactionKey = TRANS_KEY
    request.customerProfileId = str(profile_id)
    request.unmaskExpirationDate = True

    controller = getCustomerProfileController(request)
    controller.setenvironment(ENV)
    controller.execute()
    response = controller.getresponse()

    if response.messages.resultCode != "Ok":
        return None

    return response.profile

# --- Process a single profile and return expired card data ---
def process_profile(pid):
    """Process a single profile and return data if it has expired cards"""
    try:
        profile = fetch_profile(pid)
        if profile is None:
            return None

        payment_profiles = getattr(profile, 'paymentProfiles', None)
        if payment_profiles is None:
            return None
            
        expired_cards = []
        for payment in payment_profiles:
            cc = getattr(payment.payment, "creditCard", None)
            if cc is not None and is_expired(cc.expirationDate):
                expired_cards.append({
                    "paymentProfileId": str(payment.customerPaymentProfileId),
                    "cardNumber": str(cc.cardNumber),
                    "expirationDate": str(cc.expirationDate)
                })

        if expired_cards:
            return {
                "customerProfileId": str(profile.customerProfileId),
                "merchantCustomerId": str(getattr(profile, "merchantCustomerId", "")),
                "email": str(getattr(profile, "email", "")),
                "description": str(getattr(profile, "description", "")),
                "expired_cards": expired_cards
            }
        return None
    except Exception as e:
        return None

# --- Load progress from previous run ---
def load_progress():
    """Load the last processed index from progress file"""
    if os.path.exists(PROGRESS_FILE):
        with open(PROGRESS_FILE, "r") as f:
            return int(f.read().strip())
    return 0

# --- Save progress ---
def save_progress(index):
    """Save current progress to file"""
    with open(PROGRESS_FILE, "w") as f:
        f.write(str(index))

# --- Append results to CSV ---
def append_to_csv(rows, write_header=False):
    """Append rows to CSV file"""
    mode = "w" if write_header else "a"
    with open(CSV_FILE, mode, newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        if write_header:
            writer.writerow(["customerProfileId", "merchantCustomerId", "email", "description", "paymentProfileId", "cardNumber", "expirationDate"])
        for r in rows:
            for card in r["expired_cards"]:
                writer.writerow([
                    r["customerProfileId"],
                    r["merchantCustomerId"],
                    r["email"],
                    r["description"],
                    card["paymentProfileId"],
                    card["cardNumber"],
                    card["expirationDate"]
                ])

# --- Main process ---
def main():
    print("Starting expired credit card scan with INCREMENTAL SAVE...")
    print(f"Progress will be saved every {BATCH_SIZE} profiles.")
    print(f"You can stop and restart this script at any time!\n")
    
    profile_ids = fetch_all_profile_ids()
    total_profiles = len(profile_ids)
    
    # Check if we're resuming from a previous run
    start_index = load_progress()
    if start_index > 0:
        print(f"RESUMING from profile {start_index}/{total_profiles}")
        print(f"Already processed: {start_index} profiles\n")
        write_header = False
    else:
        print(f"STARTING NEW SCAN of {total_profiles} profiles\n")
        write_header = True
    
    total_expired_found = 0
    
    # Process in batches
    for batch_start in range(start_index, total_profiles, BATCH_SIZE):
        batch_end = min(batch_start + BATCH_SIZE, total_profiles)
        batch_ids = profile_ids[batch_start:batch_end]
        
        print(f"Processing batch: {batch_start} to {batch_end} ({len(batch_ids)} profiles)...")
        
        rows = []
        processed = 0
        
        # Process this batch concurrently
        with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
            future_to_pid = {executor.submit(process_profile, pid): pid for pid in batch_ids}
            
            for future in as_completed(future_to_pid):
                processed += 1
                
                if processed % 500 == 0:
                    print(f"  Batch progress: {processed}/{len(batch_ids)}... ({len(rows)} expired found in batch)")
                
                result = future.result()
                if result:
                    rows.append(result)
        
        # Save this batch to CSV
        if rows:
            append_to_csv(rows, write_header=write_header)
            write_header = False  # Only write header once
            total_expired_found += len(rows)
            print(f"✓ Saved {len(rows)} customers with expired cards from this batch")
        
        # Save progress
        save_progress(batch_end)
        print(f"✓ Progress saved: {batch_end}/{total_profiles} profiles completed")
        print(f"✓ Total expired cards found so far: {total_expired_found}\n")
    
    # Clean up progress file when complete
    if os.path.exists(PROGRESS_FILE):
        os.remove(PROGRESS_FILE)
    
    print(f"\n{'='*60}")
    print(f"SCAN COMPLETE!")
    print(f"Total profiles scanned: {total_profiles}")
    print(f"Customers with expired cards: {total_expired_found}")
    print(f"CSV saved to: {CSV_FILE}")
    print(f"{'='*60}")

if __name__ == "__main__":
    main()